In [73]:
from SuperHelperForTrainData import PDFDetails

<h2>PDF Path</h2>

In [96]:
pdf_path = "/Users/rohanraval/Downloads/RelevantCollegeNotes/JupyterNBs/lec03.pdf"
#pdf_path = "/Users/rohanraval/Desktop/Summer_Internship/nlp/NewsDataCollection/javanotes5.pdf"
pdf_path = "/Users/rohanraval/Downloads/reference(1).pdf"
file_name = "cheatsheet"

In [97]:
obj = PDFDetails(pdf_path)

In [98]:
obj.start_stop_page()

In [99]:
pdf_df = obj.get_pd()

In [100]:
pdf_df = pdf_df[pdf_df['text'].apply(lambda x: isinstance(x, str))]

<h2>Predicting</h2>

In [101]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
class SimplifiedTopicContentModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout):
        super(SimplifiedTopicContentModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.font_size_proj = nn.Linear(1, embedding_dim)
        self.context_proj = nn.Linear(1, embedding_dim)  # For contextual features
        self.highlight_proj = nn.Linear(1, embedding_dim)  # For the highlight feature
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.output_linear = nn.Linear(hidden_dim, 2)  # 2 output classes (Topic, Content)
    
    def forward(self, text_input, font_sizes, contextual_features, highlight_features):
        embeds = self.embedding(text_input)
        
        # Expand the input to match the embedding dimension
        font_size_embeds = self.font_size_proj(font_sizes).expand(-1, -1, embeds.size(-1))
        context_embeds = self.context_proj(contextual_features).expand(-1, -1, embeds.size(-1))
        highlight_embeds = self.highlight_proj(highlight_features).expand(-1, -1, embeds.size(-1))
        
        # Combine all embeddings
        combined_embeds = embeds + font_size_embeds + context_embeds + highlight_embeds
        
        lstm_out, _ = self.lstm(combined_embeds)
        logits = self.output_linear(lstm_out)
        return logits

def create_contextual_features(sizes):
    relative_sizes = [0] + [sizes[i] - sizes[i-1] for i in range(1, len(sizes))]
    max_diff = max(abs(x) for x in relative_sizes) if relative_sizes else 1
    normalized_relative_sizes = [x / max_diff for x in relative_sizes]
    return torch.tensor(normalized_relative_sizes).unsqueeze(1).float()


def predict_with_highlight(model, texts, sizes, highlighted, tokenizer, sequence_length):
    model.eval()
    tokenized_inputs = tokenizer(texts, padding=True, truncation=True, max_length=sequence_length, return_tensors="pt")
    
    font_size_tensor = torch.tensor(sizes).unsqueeze(1).float()
    highlight_tensor = torch.tensor(highlighted).unsqueeze(1).float()

    font_sizes_padded = font_size_tensor.unsqueeze(1).expand(-1, sequence_length, -1)
    highlight_features_padded = highlight_tensor.unsqueeze(1).expand(-1, sequence_length, -1)
    contextual_features_padded = create_contextual_features(sizes).unsqueeze(1).expand(-1, sequence_length, -1)
    
    with torch.no_grad():
        logits = model(tokenized_inputs['input_ids'], font_sizes_padded, contextual_features_padded, highlight_features_padded)
        predictions = torch.argmax(logits, dim=2)
    
    return predictions

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [102]:
model = torch.load('RR10DemoModel.pth')
model.eval()

SimplifiedTopicContentModel(
  (embedding): Embedding(30522, 128)
  (font_size_proj): Linear(in_features=1, out_features=128, bias=True)
  (context_proj): Linear(in_features=1, out_features=128, bias=True)
  (highlight_proj): Linear(in_features=1, out_features=128, bias=True)
  (lstm): LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.2)
  (output_linear): Linear(in_features=256, out_features=2, bias=True)
)

In [103]:
prediction_texts = list(pdf_df.get("text"))
prediction_font_sizes = list(pdf_df.get("font_sizes"))
prediction_highlighted = list(pdf_df.get("is_bold"))

In [104]:
import time

# Tokenize the prediction text
prediction_tokenized_inputs = tokenizer(prediction_texts, padding=True, truncation=True, return_tensors="pt")
prediction_sequence_length = prediction_tokenized_inputs['input_ids'].size(1)

# Prepare features
prediction_font_size_tensor = torch.tensor(prediction_font_sizes).unsqueeze(1).float()
prediction_highlight_tensor = torch.tensor(prediction_highlighted).unsqueeze(1).float()

# Expand to match sequence length
prediction_font_sizes_padded = prediction_font_size_tensor.unsqueeze(1).expand(-1, prediction_sequence_length, -1)
prediction_highlight_features_padded = prediction_highlight_tensor.unsqueeze(1).expand(-1, prediction_sequence_length, -1)
prediction_contextual_features_padded = create_contextual_features(prediction_font_sizes).unsqueeze(1).expand(-1, prediction_sequence_length, -1)

# Make predictions
prediction_logits = model(prediction_tokenized_inputs['input_ids'], prediction_font_sizes_padded, prediction_contextual_features_padded, prediction_highlight_features_padded)
prediction_classes = torch.argmax(prediction_logits, dim=2)

test_answer = []

# Function to interpret predictions
def interpret_predictions(texts, predictions):
    for i, text in enumerate(texts):
        predicted_class = predictions[i].numpy()
        if 1 in predicted_class:
            test_answer.append(1)
        else:
            test_answer.append(0)

# Interpret the predictions
start = time.time()
interpret_predictions(prediction_texts, prediction_classes)
print(time.time() - start)

0.0018737316131591797


In [105]:
pdf_df = pdf_df.assign(Title = test_answer)

<h2>Grouping the Topic - Content</h2>

In [106]:
from TopicContentRelater import TopicContent

In [107]:
obj = TopicContent(pdf_df)

In [108]:
topic_content = obj.get_dict()

In [109]:
counter = 1
for topic in topic_content:
    print(f"{counter}) {topic}")
    print("\n")
    print("\n".join(topic_content[topic]))
    print("\n\n")
    counter += 1

1) DSC 10 Reference Sheet


Below, df is a DataFrame, ser is a Series, arr is an array, babypandas has been imported as bpd, and numpy has been imported
as np.



2) Building and Organizing DataFrames


Each function/method below creates a new DataFrame. Remember to
save it!



3) Accessing Data


df.shape[0] and df.shape[1]
The number of rows and the number of columns, respectively.
bpd.DataFrame()
Creates an empty DataFrame.
bpd.read_csv(path_to_file)
Creates a DataFrame by reading from a CSV ﬁle.
df.assign(name_of_column=column_data)
Adds/replaces a column. name_of_column should not have quotes
or spaces.
df.drop(columns=column_name or [col_1_name, ..., col_k_name])
Drops a single column, or every column in a list of column names.
df.set_index(column_name)
Moves a column to the index.
df.reset_index()
Moves the index to a column.
df.get(column_name)
One column, as a Series.
df.get([col_1_name, ..., col_k_name])
Several columns, as a DataFrame.
ser.loc[label]
An element of ser, acces

In [88]:
from IPython.display import Markdown
#counter = 1
#for topic in obj.get_topics():
 #   print(f"{counter}) {topic}")
  #  counter += 1
Markdown(obj.get_index())


 lec03
# 1 Lecture 3 – Strings, Lists, and Arrays
### 1.1.1 Announcements
### 1.1.2 Resources ￿
### 1.1.3 Agenda
## 1.2 Data types
### 1.2.1 int and float
### 1.2.2 int
### 1.2.3 float
### 1.2.4 The pitfalls of float
### 1.2.5 Converting between int and float
## 1.3 Strings ￿
### 1.3.1 Strings ￿
### 1.3.2 String arithmetic
### 1.3.3 String methods
### 1.3.4 Type conversion to and from strings
### 1.3.5 Concept Check ￿ – Answer at cc.dsc10.com
## 1.4 Means and medians
### 1.4.1 Describing numerical data
### 1.4.2 The mean (i.e. average)
### = 6.
### 1.4.3 The median
### 1.4.4 Mean vs. median
### 1.4.5 Activity
## 1.5 Lists
### 1.5.1 Average temperature for a week
### 1.5.2 Lists in Python
### 1.5.3 Lists make working with sequences easy!
### 1.5.4 Types
### 1.5.5 There’s a problem…
## 1.6 Arrays
### 1.6.1 NumPy
### 1.6.2 Arrays
### 1.6.3 Positions
### 1.6.4 Accessing elements by position
### 1.6.5 Types
### 1.6.6 Array-number arithmetic
### 1.6.7 Element-wise arithmetic
## 1.7 Summary, next time
### 1.7.1 Summary
### 1.7.2 Next time

<h2> Using GenAI to produce content & writing it to a File </h2>

In [90]:
from MarkdownConverter import html
import GenAI
obj_html = html(file_name)

In [91]:
obj_html.write(obj.get_index())

In [93]:
adjoin = ""
for topic in topic_content:
    title = topic
    if len(topic_content[topic]) == 0:
        adjoin += "\n" + title
    else:
        title += adjoin
        adjoin = ""
        
    content = " ".join(topic_content[topic])
    generated_text = GenAI.generate(title, content)
    #generated_text = GenAI.google_generate(title, content)
    if generated_text != "Empty":
        generated_text = f"#{topic}" + generated_text
        obj_html.write(generated_text)

In [94]:
obj_html.get_file()
print("File Created!")

File Created!


In [111]:
obj_html.html_snippets

['\n        <div class="content-section">\n            <p>lec03</p>\n<h1>1 Lecture 3 – Strings, Lists, and Arrays</h1>\n<h3>1.1.1 Announcements</h3>\n<h3>1.1.2 Resources \uffff</h3>\n<h3>1.1.3 Agenda</h3>\n<h2>1.2 Data types</h2>\n<h3>1.2.1 int and float</h3>\n<h3>1.2.2 int</h3>\n<h3>1.2.3 float</h3>\n<h3>1.2.4 The pitfalls of float</h3>\n<h3>1.2.5 Converting between int and float</h3>\n<h2>1.3 Strings \uffff</h2>\n<h3>1.3.1 Strings \uffff</h3>\n<h3>1.3.2 String arithmetic</h3>\n<h3>1.3.3 String methods</h3>\n<h3>1.3.4 Type conversion to and from strings</h3>\n<h3>1.3.5 Concept Check \uffff – Answer at cc.dsc10.com</h3>\n<h2>1.4 Means and medians</h2>\n<h3>1.4.1 Describing numerical data</h3>\n<h3>1.4.2 The mean (i.e. average)</h3>\n<h3>= 6.</h3>\n<h3>1.4.3 The median</h3>\n<h3>1.4.4 Mean vs. median</h3>\n<h3>1.4.5 Activity</h3>\n<h2>1.5 Lists</h2>\n<h3>1.5.1 Average temperature for a week</h3>\n<h3>1.5.2 Lists in Python</h3>\n<h3>1.5.3 Lists make working with sequences easy!</h3>\n<h3